In [24]:
import keras
import numpy as np
from keras.models import Model
from keras.layers import Dense, Input, Add, MaxPooling2D, Conv2D, AveragePooling2D, Activation
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [5]:
datagen = ImageDataGenerator(
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='nearest')

In [9]:
train_gen = datagen.flow_from_directory('dogscats/sample/train',
                                       target_size=(150,150),
                                       batch_size=32,
                                       class_mode='binary')

valid_gen = datagen.flow_from_directory('dogscats/sample/valid',
                                       target_size=(150,150),
                                       batch_size=32,
                                       class_mode='binary')

Found 16 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


In [7]:
!ls dogscats

models	sample	test1  train  valid


In [8]:
img = load_img('dogscats/sample/train/cats/cat.11737.jpg')
arr = img_to_array(img)
arr = arr.reshape((1,)+arr.shape)

In [11]:
arr.shape

(1, 375, 499, 3)

In [12]:
data = np.random.randint(2,size=(1000,100))
labels = np.random.randint(2, size=(1000,1))


In [4]:
data.shape[1:]

(100,)

In [29]:
l = Input((3, 150, 150))
pool = MaxPooling2D(pool_size=(2,2))

In [30]:
def resnet_block(conv_val, input_layer, repeat_num):
    input_layer = pool(input_layer)
    conv = Conv2D(conv_val, (3,3), padding='same')
    input_layer_tmp = conv(input_layer)
    for i in xrange(repeat_num-1):
        input_layer_tmp = conv(input_layer_tmp)
    input_layer = Add()([input_layer,input_layer_tmp])
    return Activation('relu')(input_layer)

In [49]:
x = pool(l)
x = Conv2D(64, (7,7), padding='same')(x)

x = pool(x)
conv = Conv2D(64, (3,3), padding='same')
xtmp = conv(x)
xtmp = conv(xtmp)
x = Add()([x,xtmp])
x = Activation('relu')(x)

x = pool(x)
conv = Conv2D(128, (3,3), padding='same')
xtmp = conv(x)
print type(xtmp)
xtmp = conv(xtmp)
# x = Add()([x,xtmp])
# x = Activation('relu')(x)

# x = pool(x)
# conv = Conv2D(256, (3,3), padding='same')
# xtmp = conv(x)
# xtmp = conv(xtmp)
# x = Add()([x,xtmp])
# x = Activation('relu')(x)

# x = pool(x)
# conv = Conv2D(512, (3,3), padding='same')
# xtmp = conv(x)
# xtmp = conv(xtmp)
# x = Add()([x,xtmp])
# x = Activation('relu')(x)

x = AveragePooling2D(pool_size=(2,2))(x)
x = Dense(1, activation='sigmoid')(x)

<class 'theano.tensor.var.TensorVariable'>


ValueError: Input 0 is incompatible with layer conv2d_66: expected axis -1 of input shape to have value 64 but got shape (None, 0, 18, 128)

In [34]:
model = Model(l,x)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
early = keras.callbacks.EarlyStopping(monitor='val_acc',patience=45, verbose=1, mode='auto')

In [29]:
model.fit(data, labels, epochs=10, batch_size=32, callbacks=[early])

Epoch 1/10
1000/1000 [==============================] - 0s 21us/step - loss: 0.7272 - acc: 0.4810
Epoch 2/10
1000/1000 [==============================] - 0s 18us/step - loss: 0.6974 - acc: 0.5240
Epoch 3/10
1000/1000 [==============================] - 0s 19us/step - loss: 0.6831 - acc: 0.5330
Epoch 4/10
1000/1000 [==============================] - 0s 20us/step - loss: 0.6760 - acc: 0.5660
Epoch 5/10
1000/1000 [==============================] - 0s 20us/step - loss: 0.6633 - acc: 0.6010
Epoch 6/10
1000/1000 [==============================] - 0s 17us/step - loss: 0.6574 - acc: 0.6110
Epoch 7/10
1000/1000 [==============================] - 0s 16us/step - loss: 0.6469 - acc: 0.6110
Epoch 8/10
1000/1000 [==============================] - 0s 19us/step - loss: 0.6414 - acc: 0.6420
Epoch 9/10
1000/1000 [==============================] - 0s 22us/step - loss: 0.6336 - acc: 0.6310
Epoch 10/10
1000/1000 [==============================] - 0s 19us/step - loss: 0.6253 - acc: 0.6640


/home/tommy/anaconda2/lib/python2.7/site-packages/keras/callbacks.py:493: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: acc,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


In [ ]:
model.fit(data, labels, )